In [1]:
from sklearn.datasets import make_regression, make_classification, make_blobs
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import time
from GP_Classes import GaussianProcess, SparseGaussianProcess
import matplotlib.pyplot as plt
import numpy as np
import psutil
import time
import pandas as pd
import jax.numpy as jnp

In [2]:
def estimate_memory_usage(n_samples, n_features, model_type, n_inducing=None):
    # Memory for dataset (X_train and y_train)
    dataset_memory = n_samples * n_features * 8 + n_samples * 8  # X_train + y_train (float64, 8 bytes per float)
    if model_type == "GPR":
        # Full GP: Covariance matrix is N x N
        covariance_matrix_memory = n_samples ** 2 * 8  # N x N matrix, 8 bytes per float
    elif model_type == "SGPR":
        # Sparse GP: Covariance matrix is M x M (inducing points)
        if n_inducing is None:
            raise ValueError("n_inducing must be provided for SGPR")
        covariance_matrix_memory = n_inducing ** 2 * 8  # M x M matrix, 8 bytes per float
    
    # Memory for kernel parameters and gradients (assume around 1 KB for kernel + gradients)
    kernel_and_gradients_memory = 1024  # Rough estimation

    # Total memory usage estimate
    total_memory = dataset_memory + covariance_matrix_memory + kernel_and_gradients_memory

    return total_memory / (1024 ** 2)  # Convert to MB


In [3]:
# Function for farthest point sampling to select inducing points:
def farthest_point_sampling(X, n_inducing_points):
    
    # Start by selecting a random point from X as the first inducing point
    Z = [X[np.random.choice(len(X))]]
    
    # Iteratively select the farthest point from the set Z and and to the set
    for _ in range(n_inducing_points - 1):
        # Calculate distances from each point in X to the nearest point in Z
        dists = jnp.min(jnp.array([jnp.linalg.norm(X - z, axis=1) for z in Z]), axis=0)
        # Select the point farthest from Z and add it to the set
        next_point = X[jnp.argmax(dists)]
        Z.append(next_point)
    
    return jnp.array(Z)

# Function to train a GP model and return the RMSE, average uncertainty, time taken, and iteration count
def train_and_evaluate_gp(n_samples, model, optim_method):
    # Generate synthetic regression data
    X, y = make_regression(n_samples=n_samples, n_features=1, noise=1, random_state=0)  # n_features=1 for plotting
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Measure the time taken to optimize (which includes matrix inversion)
    start_time = time.time()
    
    # Optimize the model's parameters with a callback to count iterations
    if model == "GPR":
        model = GaussianProcess(X_train, y_train, optim_method)
    if model == "SGPR":
        n_inducing_points = int(len(X_train)*0.1)  # or any other number less than n_points
        # Z_train = X_train[np.random.choice(len(X_train), n_inducing_points, replace=False)]
        Z_train = farthest_point_sampling(X_train, n_inducing_points)
        model = SparseGaussianProcess(X_train, y_train,Z_train, optim_method)
    
    noise = model.fit()
    
    
    end_time = time.time()
    
    # Time taken for matrix inversion and optimization
    time_taken = end_time - start_time
    
    # Retrieve and display parameter history if optimizer is ADAM
    if hasattr(model, "adam_optimizer"):
        param_history = model.adam_optimizer.param_history
        print("Parameter History:")
        #for params in param_history:

    
    # Make predictions on the test set
    means, variances = zip(*(model.predict(x.reshape(-1, 1)) for x in X_test))

    # Compute RMSE
    rmse = np.sqrt(np.mean((np.array(means) - y_test) ** 2))
    avg_uncertantiy = np.mean(np.sqrt(np.array(variances)))
    # Calculate the lower and upper bounds of the confidence interval
    lower_bound = means - 1.96 * np.sqrt(variances)
    upper_bound = means + 1.96 * np.sqrt(variances)
    # Check if y_test is within the confidence interval bounds
    uncertainties = np.where((lower_bound <= y_test) & (y_test <= upper_bound), 1, 0)
    in_interval = np.mean(uncertainties)

    return rmse, avg_uncertantiy, time_taken, noise, in_interval

In [4]:

# Utility to get memory usage
def get_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss / (1024 ** 2)  # Return memory usage in MB

# General function to run comparison for multiple GP models
def run_comparison(max_samples, models_kernels):
    df = pd.DataFrame(columns=['Model', 'RMSE', 'Dataset Size', 'Time', 'Memory Usage', 'Avg Uncertainty', 'Predictive Noise', 'In Interval'])
    start = 50
    step = 25
    dataset_sizes = np.arange(start, max_samples + 1, step)

    # Dictionary to store results for each model
    results = {
        'loss_history': {},
        'times': {},
        'uncertainties': {},
        'memory_usages': {},
    }

    # Loop over each model and its kernel
    for model_name, optim_method in models_kernels:
        loss_history = []
        times = []
        uncertainties = []  
        memory_usages = []
        
        for idx, size in enumerate(dataset_sizes):
            rmse, avg_uncertantiy, time_taken, pred_noise, in_interval = train_and_evaluate_gp(size, model_name, optim_method)
            loss_history.append(rmse)
            memory = estimate_memory_usage(n_samples=size, n_features=1, model_type=model_name, n_inducing=0.1*size)
            uncertainties.append(avg_uncertantiy)
            times.append(time_taken)
            memory_usages.append(memory)
            name = model_name + " " + optim_method
            print(f"Model: {name}, Dataset size: {size}, Time: {time_taken:.4f} seconds, "
                  f"Memory: {memory:.2f} MB,"
                  f"Uncertainty: {avg_uncertantiy:.4f}")
            
            # Update dataframe
            new_row = pd.DataFrame({
                'Model': name,
                'Dataset Size': size,
                'RMSE': rmse,
                'Time': time_taken,
                'Memory Usage': memory,
                'Avg Uncertainty': avg_uncertantiy,
                'Predictive Noise': pred_noise,
                'In Interval': in_interval,
            }, index=[0])    

            df = pd.concat([df, new_row], ignore_index=True)
        
        # Store results for this model
        results['times'][name] = times
        results['uncertainties'][name] = uncertainties
        results['memory_usages'][name] = memory_usages
        results['loss_history'][name] = loss_history
    # Plotting the results

    print(df.head())
    df.to_csv('results_albin.csv')
    plt.figure(figsize=(10, 12))
    # Loss plot

    plt.subplot(4, 1, 1)
    for name in results['loss_history']:
        plt.plot(dataset_sizes, results['loss_history'][name], 'o-', label=f'{name} Loss')
    plt.title('Performance Metrics vs Dataset Size')
    plt.ylabel('Loss (RMSE)')
    plt.xscale('log')
    plt.grid(True)
    plt.legend()
    
    # Time Plot
    plt.subplot(4, 1, 2)
    for name in results['times']:
        plt.plot(dataset_sizes, results['times'][name], 'o-', label=f'{name} Time')
    plt.title('Performance Metrics vs Dataset Size')
    plt.ylabel('Time (seconds)')
    plt.xscale('log')
    plt.grid(True)
    plt.legend()

    # Memory Usage Plot
    plt.subplot(4, 1, 3)
    for name in results['memory_usages']:
        plt.plot(dataset_sizes, results['memory_usages'][name], 'o-', label=f'{name} Memory Usage')
    plt.ylabel('Memory Usage (MB)')
    plt.xscale('log')
    plt.grid(True)
    plt.legend()

    # Uncertainty Plot
    plt.subplot(4, 1, 4)
    for name in results['uncertainties']:
        plt.plot(dataset_sizes, results['uncertainties'][name], 'o-', label=f'{name} Avg. Uncertainty')
    plt.ylabel('Uncertainty')
    plt.xscale('log')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

    return results

In [5]:
models_kernels = [
    ("GPR", "L-BFGS-B"),
    ("SGPR", "L-BFGS-B"),
    ("GPR", "CG"),
    ("SGPR", "CG"),
    ("GPR", "ADAM"),
    ("SGPR", "ADAM"),
]
# Run the comparison for these models
run_comparison(max_samples=130, models_kernels=models_kernels)

KeyboardInterrupt: 